In [1]:
import random
import heapq
from numpy.linalg import cholesky
import numpy as np
from collections import deque
from tetris_utils import *
from tetris_reinforcement_learner import TetrisReinforcementLearner
from tetris import *
from copy import deepcopy
import pygame,sys
import csv
import os
import matplotlib.pyplot as plt
from tetris import TetrisApp
App=TetrisApp()
from tetris_reinforcement_learner import TetrisReinforcementLearner
RL_learner=TetrisReinforcementLearner(App)

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

done = False
batch_size = 40


state_size = 270
action_size = 30
# agent.load("./save/cartpole-dqn.h5")
App.init_game()
def pixalize_stone(stone):
    container=np.zeros((2,10))
    if len(stone)==1:
        container[0][...,3:3+len(stone[0])]=stone[0]
    elif stone[0][0]==7:
        container[0][...,4:4+len(stone[0])]=stone[0]
        container[1][...,4:4+len(stone[0])]=stone[1]
    else:
        container[0][...,3:3+len(stone[0])]=stone[0]
        container[1][...,3:3+len(stone[0])]=stone[1]
    return container
    
def preprocess_data(state):
    
    board=np.array(state['board'])
    stone=state['stone']
    next_stone=state['next_stone']
    pixal_stone=pixalize_stone(stone)
    pixal_next_stone=pixalize_stone(next_stone)

    final_board=np.append(pixal_next_stone,pixal_stone,axis=0)
    final_board=np.append(final_board,board,axis=0)
    final_board=np.where(final_board>0,1,0)
#     print(final_board)
    final_board=final_board.flatten()
    final_board=final_board.reshape(1,270)
    return final_board


def update_new(reinforcement_learner, old_state,new_state):## Taking holes into account
    change_in_pile_height = reinforcement_learner.get_pile_height(new_state) - reinforcement_learner.get_pile_height(old_state)
    reward = -1 * change_in_pile_height
    return reward# container=np.zeros((len(stone),10))
# for i in range(len(stone)):
#     container[i][...,3:3+len(stone[0])]=stone[i]
# board=board.flatten()
# board=board[:,np.newaxis]
# board = np.where(board>0,1,0)
# stone_idx=find_stone_idx(stone)
# next_stone_idx=find_stone_idx(state['next_stone'])

In [3]:
def get_feature(state):
    height=RL_learner.get_pile_height(state)
    holes=RL_learner.get_holes(state)
    contours=RL_learner.get_contours(state)
    features=np.array([[height],[holes],[contours]])
    return features
def get_current_state_CEM_value(state,action_sequence):
    successor_state=RL_learner.get_successor_state(state,action_sequence)
    features=get_feature(successor_state)
    for feature, value in features.items():
        CEM_val+=RL_learner.CEM_weights[feature]*value
        
def get_following_state_CEM_value(state,action_sequence,cur_theta):
    
    successor_state=RL_learner.get_successor_state(state,action_sequence)
    done=successor_state['gameover']
    if not done:
        
        features=get_feature(successor_state)

    return cur_theta.T.dot(features)

In [4]:
def get_sample_theta(mu,sigma,num_sample):
    R = cholesky(sigma)
    s = np.dot(np.random.randn(num_sample, 3), R) + mu.T
    return s

statistic=[]

## Initial search for the theta


In [8]:

top_5_theta = deque(maxlen=5)
mu=np.zeros((3,1)) #initialize mu
sigma=100*np.eye(3,3)+5 #initialize theta
s=get_sample_theta(mu,sigma,100)

num_sample=100
theta_grade=[] # store the grade each theta earned
for theta_idx in range(num_sample):

    App.init_game()
    if not App.pygame_initted: App.init_pygame()
    App.gameover=False
    App.paused=False
    App.start_game()
    done=False
    while not done:
        App.display_board()
        state=RL_learner.capture_state_attributes(App)
        dont_burn_my_cpu=pygame.time.Clock()  

        cur_theta=np.array(s[theta_idx])
        cur_theta=cur_theta[:,np.newaxis]

        ## calculate the next state value, which is store in the list: next_val
        next_val=[] # store the next_state_value
        legal_action_sequences=RL_learner.get_legal_action_sequences(state)
        for actions in legal_action_sequences:
            next_val.append(get_following_state_CEM_value(state,actions,cur_theta))
        next_action=legal_action_sequences[np.argmax(next_val)]
        App.play_action_sequence(next_action)
        new_state=RL_learner.capture_state_attributes(App)
        done=new_state['gameover']
        if done:
            theta_grade.append(App.lines)

top_5_theta_idx=list(map(theta_grade.index, heapq.nlargest(5, theta_grade)))
for idx in top_5_theta_idx:
    top_5_theta.append(s[idx])

In [12]:

np.savetxt("top_five_theta_for_CEM_3_features.txt",top_5_theta)

In [13]:
top_5_load_theta= np.loadtxt('top_five_theta_for_CEM_3_features.txt')
statistic1=[]
top_5_theta = deque(maxlen=15)
top_5_theta=top_5_load_theta
top_5_theta=top_5_theta.tolist()

In [15]:
for i in range(10000):
    np_top_5_theta=np.array(top_5_theta).T
    mu=np.mean(np_top_5_theta,axis=1)
    sigma=np.cov(np_top_5_theta)+max(5-(i+1)/10,0)*np.eye(3)
    s=get_sample_theta(mu,sigma,50)

    num_sample=50
    theta_grade=[] # store the grade each theta earned
    for theta_idx in range(num_sample):
        print('iter: ',i,', idx: ',theta_idx,end="\r")
        App.init_game()
        if not App.pygame_initted: App.init_pygame()
        App.gameover=False
        App.paused=False
        App.start_game()
        done=False
        while not done:
            App.display_board()
            state=RL_learner.capture_state_attributes(App)
            dont_burn_my_cpu=pygame.time.Clock()  

            cur_theta=np.array(s[theta_idx])
            cur_theta=cur_theta[:,np.newaxis]

            ## calculate the next state value, which is store in the list: next_val
            next_val=[] # store the next_state_value
            legal_action_sequences=RL_learner.get_legal_action_sequences(state)
            for actions in legal_action_sequences:
                next_val.append(get_following_state_CEM_value(state,actions,cur_theta))
            next_action=legal_action_sequences[np.argmax(next_val)]
            App.play_action_sequence(next_action)
            new_state=RL_learner.capture_state_attributes(App)
            done=new_state['gameover']
            if done:
                theta_grade.append(App.lines)

    top_5_theta_idx=list(map(theta_grade.index, heapq.nlargest(5, theta_grade)))
    for idx in top_5_theta_idx:
        top_5_theta.append(s[idx])
        
    statistic1.append([np.max(theta_grade),np.min(theta_grade),np.mean(theta_grade)])
    print(i)
    print(statistic1)

In [296]:
statistic_for_non_noise=statistic
statistic_for_non_noise

[[355, 5, 10],
 [1455, 5, 255],
 [2119, 8, 387.67],
 [3036, 7, 549.47],
 [4638, 3, 676.15],
 [3499, 38, 807.28],
 [2729, 225, 1446.6],
 [2493, 186, 758.25],
 [2000, 152, 618.875],
 [2021, 248, 1010.25],
 [2152, 114, 864.125],
 [2428, 86, 863.875],
 [1718, 25, 666.0],
 [2167, 95, 862.875],
 [2196, 200, 795.75],
 [2288, 82, 634.0],
 [1661, 101, 851.625],
 [2108, 434, 1062.0]]